In [1]:
import os
import sys
root_dir = os.path.split(os.getcwd())[0]
project_dir = os.path.join(root_dir, 'python')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import subprocess
output = subprocess.Popen(f"cd {project_dir} && pipenv --venv", shell=True, stdout=subprocess.PIPE).stdout.read().decode('utf-8')
if len(output) > 0:
    # Include the pipenv in this notebook
    pipenv_path = os.path.join(output[:-1], 'lib', 'python3.6', 'site-packages')
    if pipenv_path not in sys.path:
        sys.path.append(pipenv_path)

In [3]:
url = "http://localhost:9090"
user = "admin"
password = "admin"

In [4]:
from pivot import Connector
from datetime import datetime

connector = Connector(url, user, password)

In [7]:
mdx_query = connector.mdx_query("""
SELECT
  NON EMPTY [Measures].[Total scores] ON COLUMNS,
  NON EMPTY [Games].[Games].[Team2Name].Members ON ROWS
  FROM [NanoPivotCube]
""")
mdx_query.dataframe.head()

,Team1Name,Team2Name,Total scores
0,Argentina,Croatia,0
1,Argentina,Iceland,2
2,Australia,Peru,0
3,Belgium,Panama,3
4,Belgium,Tunisia,0


In [9]:
aggr_df = mdx_query.dataframe.groupby('Team1Name').sum()
aggr_df.head()

,Total scores
Team1Name,
Argentina,2
Australia,0
Belgium,3
Brazil,2
Colombia,3


In [50]:
top_query = connector.mdx_query("""
SELECT
  NON EMPTY [Measures].[Total scores] ON COLUMNS,
  NON EMPTY [Games].[Games].[Team1Name].Members ON ROWS
  FROM [NanoPivotCube]
""")
top_query.dataframe.head()

,Team1Name,Team2Name,Total scores
0,Argentina,All,2
1,Australia,All,0
2,Belgium,All,3
3,Brazil,All,2
4,Colombia,All,3


In [52]:
def check_scores(team):
    v1 = aggr_df.loc[[team]]['Total scores'].values[0]
    v2 = top_query.dataframe[top_query.dataframe['Team1Name'] == team]['Total scores'].values[0]
    return v1 == v2
    
check_scores('Brazil')

True

In [54]:
check_scores('Colombia')

True